In [50]:
####################################################################################################
## Due to the limitations of the computing power of the laptop, I choose one type of category and ##
## the first 30 lines of the dataset and reduced the learning rate                                ##
## Lab_1 Bekzat Bakytbek                                                                          ##
####################################################################################################
import string
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
import re
import torch
from torch.autograd import Variable
import numpy as np
import torch.functional as F
import torch.nn.functional as F
import pandas as pd
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from string import punctuation
from nltk.stem.porter import *
from nltk.stem import WordNetLemmatizer
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

In [51]:
# fucntion that cleans the input data and prepares for manipalations
def prep_func(sentence):
    wordnet_lemmatizer = WordNetLemmatizer()
    sentence = stemmer.stem(sentence) #trim the endings
    sentence = sentence.lower() #convert everything to lowercase
    tokenizer = RegexpTokenizer(r'\w+') #  #exclude puntuations
    tokens = tokenizer.tokenize(sentence)
    filtered_words = [wordnet_lemmatizer.lemmatize(word) for word in tokens]
    filtered_words = [w for w in tokens if not w in stopwords.words('english')] #exclude stopping words
    filtered_words = [c for c in filtered_words if not c.isdigit()] #exclude the numbers
    return " ".join(filtered_words)

In [52]:
bbc_dataset = "bbc_text.csv" # bbc_dataset variable
dataset = pd.read_csv(bbc_dataset)
dataset = dataset[dataset.category == 'tech']
dataset.index = np.arange(len(dataset))
dataset = dataset[:30]
print(dataset.head())

  category                                               text
0     tech  tv future in the hands of viewers with home th...
1     tech  games maker fights for survival one of britain...
2     tech  security warning over  fbi virus  the us feder...
3     tech  halo 2 heralds traffic explosion the growing p...
4     tech  mobile audio enters new dimension as mobile ph...


In [53]:
corpus = dataset['text']
print(corpus)

0     tv future in the hands of viewers with home th...
1     games maker fights for survival one of britain...
2     security warning over  fbi virus  the us feder...
3     halo 2 heralds traffic explosion the growing p...
4     mobile audio enters new dimension as mobile ph...
5     mobile gig aims to rock 3g forget about going ...
6     firefox browser takes on microsoft microsoft s...
7     gamers snap up new sony psp gamers have bought...
8     pandas benefit from wireless net the world s d...
9     broadband in the uk growing fast high-speed ne...
10    microsoft gets the blogging bug software giant...
11    tech helps disabled speed demons an organisati...
12    loyalty cards idea for tv addicts viewers coul...
13    web radio takes spanish rap global spin the ra...
14    screensaver tackles spam websites net users ar...
15    ibm frees 500 software patents computer giant ...
16    seamen sail into biometric future the luxury c...
17    europe backs digital tv lifestyle how peop

In [54]:
tokenized_corpus = []
stemmer = PorterStemmer()
for i in range(len(corpus)):
    sentence = prep_func(corpus[i])
    tokenized_corpus.append(word_tokenize((sentence)))
print(tokenized_corpus)

LookupError: 
**********************************************************************
  Resource [93mwordnet[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('wordnet')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mcorpora/wordnet[0m

  Searched in:
    - 'C:\\Users\\Bekzat/nltk_data'
    - 'C:\\Users\\Bekzat\\Anaconda3\\nltk_data'
    - 'C:\\Users\\Bekzat\\Anaconda3\\share\\nltk_data'
    - 'C:\\Users\\Bekzat\\Anaconda3\\lib\\nltk_data'
    - 'C:\\Users\\Bekzat\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
**********************************************************************


In [55]:
dictionary = []
for sentence in tokenized_corpus:
    for token in sentence:
        if token not in dictionary:
            dictionary.append(token)

word2idx = {w: idx for (idx, w) in enumerate(dictionary)}
idx2word = {idx: w for (idx, w) in enumerate(dictionary)}

dictionary_size = len(dictionary)
print(dictionary)

[]


In [56]:
word2idx

{}

In [57]:
window_size = 2
idx_pairs = []
# loop the sentence
for sentence in tokenized_corpus:
    indices = [word2idx[word] for word in sentence]
    # loop for center word
    for cword_position in range(len(indices)):
        # for each window position
        for w in range(-window_size, window_size + 1):
            context_word_position = cword_position + w
            # restriction to be within the sentence
            if context_word_position < 0 or context_word_position >= len(indices) or cword_position == context_word_position:
                continue
            context_word_idx = indices[context_word_position]
            idx_pairs.append((indices[cword_position], context_word_idx))

idx_pairs = np.array(idx_pairs)

In [58]:
idx_pairs[:10]
print(len(idx_pairs))

0


In [59]:
#input layer function returns center words with dimensions [1, dictionary_size]
def input_layer_func(word_idx):
    x = torch.zeros(dictionary_size).float()
    x[word_idx] = 1.0
    return x


In [60]:
embedding_dims = 5
W1 = Variable(torch.randn(embedding_dims, dictionary_size).float(), requires_grad=True)
W2 = Variable(torch.randn(dictionary_size, embedding_dims).float(), requires_grad=True)
num_epochs = 101
learning_rate = 0.01

for epo in range(num_epochs):
    loss_val = 0
    for data, target in idx_pairs:
        x = Variable(input_layer_func(data)).float()
        y_true = Variable(torch.from_numpy(np.array([target])).long())

        z1 = torch.matmul(W1, x)
        z2 = torch.matmul(W2, z1)
    
        log_softmax = F.log_softmax(z2, dim=0)

        loss = F.nll_loss(log_softmax.view(1,-1), y_true)
        loss_val += loss.data.item()
        loss.backward()
        W1.data -= learning_rate * W1.grad.data
        W2.data -= learning_rate * W2.grad.data
        vec = W2.data
        #print(vec)
        W1.grad.data.zero_()
        W2.grad.data.zero_()
    if epo % 10 == 0:    
        print(f'Loss at epo {epo}: {loss_val/len(idx_pairs)}')

ZeroDivisionError: division by zero

In [65]:
# function that find nearest words
def find_nearest_func(word):
    index_of_word = -1
    for i in range(len(dictionary)):
        if  (dictionary[i] == word):
            index_of_word = i
    if (index_of_word < 0):
        print('word does not exist on dataset')
    if (index_of_word >= 0):
        print('index of the word', dictionary[index_of_word],' is ',index_of_word)
        print('task1 Thensors of words')
        print(W2.data)
        cos_distance = []
        labels = []
        aaa = np.array(W2.data)
        for j in range(len(W2)):
            result = np.dot(np.array(aaa[j]),np.array(aaa[index_of_word]))/(((np.sum(aaa[j]**2))**0.5) * ((np.sum(aaa[index_of_word]**2))**0.5))
            cos_distance.append([result, j])
        cos_distance_sorted = sorted(cos_distance, key=lambda x: x[0], reverse=True)
        for i in (range(10)):
            print(cos_distance_sorted[i], dictionary[cos_distance_sorted[i][1]])
            labels.append(dictionary[cos_distance_sorted[i][1]])
        array_for_dot_cloud = []
        array_for_dot_cloud_label = []
        for i in range(11):
            array_for_dot_cloud.append(np.array(aaa[cos_distance_sorted[i][1]]))
            array_for_dot_cloud_label.append(dictionary[cos_distance_sorted[i][1]])
        print(W2[cos_distance_sorted[i][1]], dictionary[cos_distance_sorted[i][1]])
        array_for_dot_cloud = np.array(array_for_dot_cloud)
        print(array_for_dot_cloud)
        X_embedded = TSNE(n_components=2).fit_transform(array_for_dot_cloud)
        X_embedded.shape
        #print(X_embedded)
        x = []
        y = []
        new_values = X_embedded
        for value in new_values:
            x.append(value[0])
            y.append(value[1])
        plt.figure(figsize=(10, 10)) 
        for i in range(len(x)):
            plt.scatter(x[i],y[i])
            plt.annotate(array_for_dot_cloud_label[i],
                     xy=(x[i], y[i]),
                     xytext=(5, 2),
                     textcoords='offset points',
                     ha='right',
                     va='bottom')
        plt.show()

In [66]:
find_nearest_func('america')

word does not exist on dataset
